In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from matplotlib import pyplot as plt
from sklearn import metrics
import seaborn as sns
import pandas as pd 
from sklearn.model_selection import train_test_split
import xgboost
sns.set(font_scale=1.7)
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'xgboost'

In [6]:
X_train = pd.read_csv('C:/Users/UK_W/Desktop/python/vibration/train_features.csv')
Y_train = pd.read_csv('C:/Users/UK_W/Desktop/python/vibration/train_target.csv')
X_train

,id,Time,S1,S2,S3,S4
0,0,0.000000,0.00,0.0,0.00,0.00
1,0,0.000004,0.00,0.0,0.00,0.00
2,0,0.000008,0.00,0.0,0.00,0.00
3,0,0.000012,0.00,0.0,0.00,0.00
4,0,0.000016,0.00,0.0,0.00,0.00
...,...,...,...,...,...,...
1049995,2799,0.001480,-89256.32,-164614.6,48186.29,-147910.20
1049996,2799,0.001484,-29233.44,-179404.2,109534.40,-111591.10
1049997,2799,0.001488,37147.25,-190011.7,178781.10,-86836.78
1049998,2799,0.001492,98278.50,-190771.4,245726.70,-75660.00


In [27]:
X_tr, X_vid, Y_tr, Y_vid = train_test_split(X_train3, Y_train, test_size=0.3, random_state=2018,)

In [ ]:
def ResNetBlock(x, nf, strides):
    
    shortcut = x
    
    x = layers.Conv1D(nf, 3, padding = 'same', strides = strides)(x)
#    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ELU()(x)
    x = layers.Conv1D(nf, 3, padding = 'same', strides = 1)(x)
#    x = tf.keras.layers.BatchNormalization()(x)
    
    if strides != 1:
        shortcut = layers.Conv1D(nf, 1, strides = strides)(shortcut)
#        shortcut = tf.keras.layers.BatchNormalization()(shortcut)
        
    x = layers.add([x, shortcut])
    x = tf.keras.layers.ELU()(x)
    return x 

def DenseBlock(x, nodes):
    
    shortcut = x
    
    x = tf.keras.layers.Dense(nodes, activation = 'elu')(x)
    x = tf.keras.layers.Dense(nodes, activation = 'elu')(x)
    x = layers.add([x, shortcut])
    return x 

In [3]:
#Model
tf.random.set_seed(44)

inputs = keras.Input(shape = (355,620,2), name = 'input')

x = layers.Conv2D(16, 3, activation = 'elu', padding = 'same')(inputs)
#x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2))(x)

x = layers.Conv2D(32, 3, activation = 'elu', padding = 'same')(x)
#x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2))(x)

x = layers.Conv2D(64, 3, activation = 'elu', padding = 'same')(x)
#x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2))(x)

x = layers.Conv2D(128, 3, activation = 'elu', padding = 'same')(x)
#x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2))(x)

x = layers.Conv2D(256, 3, activation = 'elu', padding = 'same')(x)
#x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2))(x)

x = layers.Conv2D(512, 3, activation = 'elu', padding = 'same')(x)

x = layers.Conv2DTranspose(256, 3, activation = 'elu', padding = 'same')(x)
x = layers.UpSampling2D((2,2))(x)

x = layers.Conv2DTranspose(128, 3, activation = 'elu', padding = 'same')(x)
x = layers.UpSampling2D((2,2))(x)

x = layers.Conv2DTranspose(64, 3, activation = 'elu', padding = 'same')(x)
x = layers.UpSampling2D((2,2))(x)

x = layers.Conv2DTranspose(32, 3, activation = 'elu', padding = 'same')(x)
x = layers.UpSampling2D((2,2))(x)

x = layers.Conv2DTranspose(3, 3, activation = 'elu', padding = 'same')(x)
x = layers.UpSampling2D((2,2))(x)

#outputs = layers.Dense(1)(x)

model = keras.Model(inputs, x, name = 'resnet')

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0012, decay = 0.013),loss='mse', metrics=['mae'])

#hist = model.fit(X_tr,Y_tr['M'],epochs=500, validation_data=(X_vid, Y_vid['M']), batch_size=32, callbacks = [callbacks,checkpoint])
#model.load_weights(modelpath)

model.summary()

Model: "resnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 355, 620, 2)]     0         
                                                                 
 conv2d (Conv2D)             (None, 355, 620, 16)      304       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 177, 310, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 177, 310, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 88, 155, 32)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 88, 155, 64)       18496

In [4]:
#Model U-net
tf.random.set_seed(44)

inputs = keras.Input(shape = (256,256,3), name = 'input')

x = layers.Conv2D(16, 3, activation = 'elu', padding = 'same')(inputs)
#x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2))(x)

x = layers.Conv2D(32, 3, activation = 'elu', padding = 'same')(x)
skip1 = x
#x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2))(x)

x = layers.Conv2D(64, 3, activation = 'elu', padding = 'same')(x)
skip2 = x
#x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2))(x)

x = layers.Conv2D(128, 3, activation = 'elu', padding = 'same')(x)
skip3 = x
#x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2))(x)

x = layers.Conv2D(256, 3, activation = 'elu', padding = 'same')(x)
#x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D(pool_size=(2))(x)

x = layers.Conv2D(512, 3, activation = 'elu', padding = 'same')(x)

x = layers.Conv2DTranspose(256, 3, activation = 'elu', padding = 'same')(x)
x = layers.UpSampling2D((2,2))(x)

x = layers.Conv2DTranspose(128, 3, activation = 'elu', padding = 'same')(x)
x = layers.UpSampling2D((2,2))(x)
x = layers.add([x, skip3])

x = layers.Conv2DTranspose(64, 3, activation = 'elu', padding = 'same')(x)
x = layers.UpSampling2D((2,2))(x)
x = layers.add([x, skip2])

x = layers.Conv2DTranspose(32, 3, activation = 'elu', padding = 'same')(x)
x = layers.UpSampling2D((2,2))(x)
x = layers.add([x, skip1])

x = layers.Conv2DTranspose(16, 3, activation = 'elu', padding = 'same')(x)
x = layers.UpSampling2D((2,2))(x)


output = layers.Conv2DTranspose(3, 3, activation = 'elu', padding = 'same')(x)

#outputs = layers.Dense(1)(x)

model = keras.Model(inputs, output, name = 'unet')

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0012, decay = 0.013),loss='mse', metrics=['mae'])

#hist = model.fit(X_tr,Y_tr['M'],epochs=500, validation_data=(X_vid, Y_vid['M']), batch_size=32)

model.summary()

Model: "unet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_6 (Conv2D)              (None, 256, 256, 16  448         ['input[0][0]']                  
                                )                                                                 
                                                                                                  
 max_pooling2d_5 (MaxPooling2D)  (None, 128, 128, 16  0          ['conv2d_6[0][0]']               
                                )                                                              

In [32]:
model.evaluate(X_vid,Y_vid['M'])

27/27 [==============================] - 0s 4ms/step - loss: 0.8955 - mae: 0.4447


[0.89546799659729, 0.44466617703437805]

In [24]:
plt.imshow(Y_tr[0,:,:,:])

<function matplotlib.pyplot.imshow(X, cmap=None, norm=None, aspect=None, interpolation=None, alpha=None, vmin=None, vmax=None, origin=None, extent=None, *, filternorm=True, filterrad=4.0, resample=None, url=None, data=None, **kwargs)>

In [ ]:
prediction = model.predict(X_tr[0,:,:,:])
plt.imshow(prediction)

In [36]:
#표준화 MAE
print('X MAE(%) : {:.5f}'.format(np.abs(Y_vid['X']-Y_vid['P_X']).mean()/800*100))
print('Y MAE(%) : {:.5f}'.format(np.abs(Y_vid['Y']-Y_vid['P_Y']).mean()/800*100))
print('M MAE(%) : {:.5f}'.format(np.abs(Y_vid['M']-Y_vid['P_M']).mean()/150*100))
print('V MAE(%) : {:.5f}'.format(np.abs(Y_vid['V']-Y_vid['P_V']).mean()/0.8*100))

X MAE(%) : 0.08454
Y MAE(%) : 0.01465
M MAE(%) : 0.29644
V MAE(%) : 0.38901


In [37]:
#표준화 MAE
print('X MAE(%) : {:.5f}'.format(np.abs(Y_vid['X']-Y_vid['P_X']).mean()/800*100))
print('Y MAE(%) : {:.5f}'.format(np.abs(Y_vid['Y']-Y_vid['P_Y']).mean()/800*100))
print('M MAE(%) : {:.5f}'.format(np.abs(Y_vid['M']-Y_vid['P_M']).mean()/150*100))
print('V MAE(%) : {:.5f}'.format(np.abs(Y_vid['V']-Y_vid['P_V']).mean()/0.8*100))

X MAE(%) : 0.08454
Y MAE(%) : 0.01465
M MAE(%) : 0.29644
V MAE(%) : 0.38901


In [38]:
#표준화 RMSE
print('X RMSE(%) : {:.5f}'.format(np.sqrt(((Y_vid['X']-Y_vid['P_X'])**2).mean()/800*100)))
print('Y RMSE(%) : {:.5f}'.format(np.sqrt(((Y_vid['Y']-Y_vid['P_Y'])**2).mean()/800*100)))
print('M RMSE(%) : {:.5f}'.format(np.sqrt(((Y_vid['M']-Y_vid['P_M'])**2).mean()/150*100)))
print('V RMSE(%) : {:.5f}'.format(np.sqrt(((Y_vid['V']-Y_vid['P_V'])**2).mean()/0.8*100)))

X RMSE(%) : 1.31683
Y RMSE(%) : 0.40263
M RMSE(%) : 0.77264
V RMSE(%) : 0.05599


In [39]:
# R2_Score
from sklearn.metrics import r2_score
print('X R2 : {:.5f}'.format(r2_score(Y_vid['X'] , Y_vid['P_X'])))
print('Y R2 : {:.5f}'.format(r2_score(Y_vid['Y'] , Y_vid['P_Y'])))
print('M R2 : {:.5f}'.format(r2_score(Y_vid['M'] , Y_vid['P_M'])))
print('V R2 : {:.5f}'.format(r2_score(Y_vid['V'] , Y_vid['P_V'])))

X R2 : 0.99978
Y R2 : 0.99998
M R2 : 0.99964
V R2 : 0.99968
